In [7]:
# 03_model_training_setup.ipynb

import os
import sys
import pandas as pd
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

# --- Proje ana dizinini Python'a tanıtma ---
PROJE_ANA_DIZINI = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if PROJE_ANA_DIZINI not in sys.path:
    sys.path.append(PROJE_ANA_DIZINI)
print(f"✅ Proje ana dizini ({PROJE_ANA_DIZINI}) Python'un arama yoluna eklendi.")

# Bu import artık sorunsuz çalışıyor
from src.data_utils import HeadCTDataset, collate_fn_skip_corrupted
print("✅ src/data_utils.py modülü başarıyla import edildi.")

# --- Ayarlar ---
# DÜZELTME: Çalışma dizini 'notebooks' olduğu için bir üst dizine çıkmamız gerekiyor.
MASTER_CSV_PATH = '../Data/processed/master_image_list.csv' 
IMAGE_SIZE = 256
BATCH_SIZE = 16

# --- Veri Yükleme ve Hazırlama ---
try:
    df_master = pd.read_csv(MASTER_CSV_PATH)

    data_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])

    dataset = HeadCTDataset(dataframe=df_master.head(2000), transform=data_transforms)

    dataloader = DataLoader(
        dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=True, 
        collate_fn=collate_fn_skip_corrupted
    )
    print("✅ Dataset ve DataLoader modül kullanılarak başarıyla oluşturuldu.")

    # --- Test ---
    images, labels = next(iter(dataloader))
    
    if images.nelement() == 0:
         print("⚠️ DataLoader boş bir batch döndürdü.")
    else:
        print(f"\n✅ Test Başarılı! Bir batch veri başarıyla çekildi.")
        print(f"   Görüntü şekli: {images.shape}")
        print(f"   Etiket şekli:  {labels.shape}")

except FileNotFoundError:
    print(f"❌ HATA: '{MASTER_CSV_PATH}' yolu bulunamadı. Lütfen dosya yapınızı kontrol edin.")
except Exception as e:
    print(f"❌ DataLoader testi sırasında hata: {e}")

✅ Proje ana dizini (c:\Users\iamem\OneDrive\Desktop\AI-Proje\Prototype2) Python'un arama yoluna eklendi.
✅ src/data_utils.py modülü başarıyla import edildi.
✅ Dataset ve DataLoader modül kullanılarak başarıyla oluşturuldu.

✅ Test Başarılı! Bir batch veri başarıyla çekildi.
   Görüntü şekli: torch.Size([16, 1, 256, 256])
   Etiket şekli:  torch.Size([16])


In [8]:
# 03_model_training_setup.ipynb notebook'unda YENİ BİR HÜCRE

# 1. Yeni model fonksiyonumuzu .py dosyasından import edelim
from src.model import get_model

# 2. Cihazı ayarlayalım (GPU varsa CUDA, yoksa CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılacak cihaz: {device}")

# 3. Modeli oluşturalım ve cihaza taşıyalım
try:
    # get_model() fonksiyonunu çağırarak ResNet18 modelini alıyoruz
    model = get_model(pretrained=True, num_classes=2).to(device)

    # DataLoader'dan daha önce aldığımız bir grup görüntüyü de cihaza gönderelim
    # Bu değişkenlerin bir önceki hücrede oluşturulmuş olması gerekir.
    images = images.to(device)
    
    # 4. Modelden bir tahmin almayı deneyelim (ileri besleme - forward pass)
    with torch.no_grad(): # Gradyan hesaplamasını kapatarak bu testi hızlandırırız
        outputs = model(images)

    print("\n✅ ResNet18 Model Testi Başarılı!")
    print(f"Modelin çıktı şekli: {outputs.shape}") # Beklenen: torch.Size([16, 2])
    print("\nÖrnek Çıktı (Logits) -> [Kanamasız Skoru, Kanamalı Skoru]:")
    # Çıktıyı CPU'ya geri alıp numpy array'ine çevirerek daha okunaklı yazdırıyoruz
    print(outputs[0].cpu().numpy()) 

except NameError:
    print("❌ HATA: 'images' değişkeni bulunamadı. Lütfen bu hücreden önceki DataLoader test hücresini çalıştırdığınızdan emin olun.")
except Exception as e:
    print(f"❌ Model testi sırasında bir hata oluştu: {e}")

Kullanılacak cihaz: cpu

✅ ResNet18 Model Testi Başarılı!
Modelin çıktı şekli: torch.Size([16, 2])

Örnek Çıktı (Logits) -> [Kanamasız Skoru, Kanamalı Skoru]:
[-0.42095518  0.19924936]


In [9]:
# 03_model_training_setup.ipynb notebook'unda TEK BİR HÜCRE

import os
import sys
import torch
import torch.nn as nn
from tqdm.notebook import tqdm

# --- 1. Proje Yolunu Python'a Tanıtma ---
# Bu blok, 'src' klasörünü bulabilmemiz için her zaman en başta olmalı.
PROJE_ANA_DIZINI = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if PROJE_ANA_DIZINI not in sys.path:
    sys.path.append(PROJE_ANA_DIZINI)
print(f"✅ Proje ana dizini ({PROJE_ANA_DIZINI}) Python'un arama yoluna eklendi.")

# --- 2. Gerekli Modülleri Import Etme ---
# Artık bu importlar sorunsuz çalışacak.
from src.model import get_model
# DataLoader'ı bir önceki hücrede oluşturduğumuzu varsayıyoruz. 
# Eğer notebook'u yeniden başlattıysanız, DataLoader'ı oluşturan hücreyi tekrar çalıştırmalısınız.
print("✅ src modülleri başarıyla import edildi.")


# --- 3. Model, Optimizer ve Kayıp Fonksiyonunu Kurma ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılacak cihaz: {device}")

# Modeli al ve cihaza gönder
model = get_model(pretrained=True, num_classes=2).to(device)

# Kayıp Fonksiyonu
loss_function = nn.CrossEntropyLoss()

# Optimizasyon Algoritması (AdamW)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

print("✅ Model, Optimizer ve Kayıp Fonksiyonu hazır.")


# --- 4. Tek Epoch'luk Eğitim Döngüsü ---
# 'dataloader' değişkeninin bir önceki hücrede tanımlandığından emin olun!
try:
    if 'dataloader' not in locals():
        raise NameError("'dataloader' değişkeni bulunamadı. Lütfen önceki hücreyi çalıştırın.")

    model.train() # Modeli eğitim moduna al
    total_loss = 0.0
    processed_batches = 0

    print("\n--- Eğitim Başlatılıyor (1 Epoch, Optimizer: AdamW) ---")

    for images, labels in tqdm(dataloader, desc="Eğitim Adımları"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        processed_batches += 1

    avg_loss = total_loss / processed_batches
    print(f"\n✅ Eğitim tamamlandı!")
    print(f"Ortalama Kayıp (Average Loss): {avg_loss:.4f}")

except Exception as e:
    print(f"❌ Eğitim sırasında bir hata oluştu: {e}")

✅ Proje ana dizini (c:\Users\iamem\OneDrive\Desktop\AI-Proje\Prototype2) Python'un arama yoluna eklendi.
✅ src modülleri başarıyla import edildi.
Kullanılacak cihaz: cpu
✅ Model, Optimizer ve Kayıp Fonksiyonu hazır.

--- Eğitim Başlatılıyor (1 Epoch, Optimizer: AdamW) ---


Eğitim Adımları:   0%|          | 0/125 [00:00<?, ?it/s]


✅ Eğitim tamamlandı!
Ortalama Kayıp (Average Loss): 0.0803
